# Derive Ratings Values from Source Data
This notebook provides a straight forward solution for gaining insight into the ratings across four different agencies. These are as follows:
- Moodys
- SP
- Fitch
- Internal Rating

This notebook takes the alphabetical values used by these agencies and converts them into a numeric format, which then allows for greater insight as to the highest and lowest ratings for a given stock across the four agencies, along with the average rating.

## Setup

In [1]:
# Import LUSID
import lusid
import lusid.models as models
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import format_instruments_response

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import pandas as pd
import json
import lusid
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")


print ('LUSID Environment Initialised')
print ('LUSID API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.6.10790.0


## Loading Data into Lusid
The intial data is loaded from a CSV file, this will then be upserted into Lusid. All of the stocks will be created as new instruments, with custom properties attached to allow for the various ratings.

In [2]:
vals = pd.read_csv("uk_stocks_with_ratings.csv")
vals.head()

,Ticker,Name,Sector,ISIN,SEDOL,Figi,Moodys,SP,Fitch,Internal
0,III LN,3i,Financial Services,GB00B1YW4409,B1YW440,BBG000BZZ876,BBB+,A+,AA-,BBB
1,ADM LN,Admiral Group,Nonlife Insurance,GB00B02J6398,B02J639,BBG000PG2GZ0,NaN,A+,A,AA
2,AAL LN,Anglo American plc,Mining,GB00B1XZS820,B1XZS82,BBG000BBLDF4,BBB+,AA,AA,NaN
3,ANTO LN,Antofagasta,Mining,GB0000456144,45614,BBG000BD4SC9,AA,A-,A+,A+
4,AHT LN,Ashtead Group,Support Services,GB0000536739,53673,BBG000BD42C6,A,AA,BBB-,BBB+


In [3]:
# Create mapping for upsert
instrument_mapping = {
    "identifiers": {
        "Isin": "ISIN",
        "Sedol": "SEDOL",
        "Figi": "Figi"
    },
    "mapping_required": {
        "name": "Name",
        "definition.instrument_type": "$Equity",
        "definition.dom_ccy": "$GBP"
    },
    "property_columns": [
        {
            "source": "Moodys",
            "target": "Moodys Rating",
            "scope": "moodys",
        },
        {
            "source": "SP",
            "target": "SP Rating",
            "scope": "sp"
        },
        {
            "source": "Fitch",
            "target": "Fitch Rating",
            "scope": "fitch"
        },
        {
            "source": "Internal",
            "target": "Internal Rating",
            "scope": "internal_r"
        }
    ]
}

In [4]:
# Upsert to lusid
response = load_from_data_frame(
    api_factory=api_factory,
    scope="default",
    data_frame=vals,
    mapping_required=instrument_mapping["mapping_required"],
    mapping_optional={},
    file_type="instrument",
    identifier_mapping=instrument_mapping["identifiers"],
    property_columns=instrument_mapping["property_columns"],
    properties_scope="ratings",
)

instrument_response_objs =  [ value.lusid_instrument_id for key, value in response['instruments']['success'][0].values.items()]
instrument_response_objs

['LUID_00003DTU',
 'LUID_00003DU1',
 'LUID_00003DTK',
 'LUID_00003GSO',
 'LUID_00003GSQ',
 'LUID_00003DVP',
 'LUID_00003GSE',
 'LUID_00003GSA',
 'LUID_00003DVX',
 'LUID_00003D8K',
 'LUID_00003GSH',
 'LUID_00003D86',
 'LUID_00003DVH',
 'LUID_00003D8F',
 'LUID_00003GST',
 'LUID_00003DVS',
 'LUID_00003DU6',
 'LUID_00003GS6',
 'LUID_00003DVJ',
 'LUID_00003GSM',
 'LUID_00003DUE',
 'LUID_00003DUV',
 'LUID_00003GSX',
 'LUID_00003DU9',
 'LUID_00003DUN',
 'LUID_00003DUX',
 'LUID_00003DUW',
 'LUID_00003DTM',
 'LUID_00003DTG',
 'LUID_00003DUR',
 'LUID_00003GSI',
 'LUID_00003DUO',
 'LUID_00003DTP',
 'LUID_00003D8E',
 'LUID_00003DUU',
 'LUID_00003GSJ',
 'LUID_00003DTX',
 'LUID_00003DVR',
 'LUID_00003DTQ',
 'LUID_00003GSL',
 'LUID_00003DTV',
 'LUID_00003GS7',
 'LUID_00003GSB',
 'LUID_00003DTF',
 'LUID_00003DUI',
 'LUID_00003GSD',
 'LUID_00003DU5',
 'LUID_00003GSP',
 'LUID_00003DUZ',
 'LUID_00003D8I',
 'LUID_00003DV1',
 'LUID_00003DUD',
 'LUID_00003GSC',
 'LUID_00003DTJ',
 'LUID_00003DVC',
 'LUID_000

## Create Derived Properties for Average Rating

In [5]:
ratings_mapping = """'AA'=90,
    'AA-'=85,
    'A+'=80,
    'A'=75,
    'A-'=70,
    'BBB+'=65,
    'BBB'=60,
    'BBB-'=55
"""

In [6]:
# Create derived properties for ratings derivations: min, max, average
# TODO can I round and then convert back to letters?
try:
    properties_response = api_factory.build(
        lusid.api.PropertyDefinitionsApi
    ).create_derived_property_definition(
        create_derived_property_definition_request=lusid.models.CreateDerivedPropertyDefinitionRequest(
            domain="Instrument",
            scope="ratings-scope",
            code="AverageRating",
            display_name="Average Rating",
            data_type_id=lusid.ResourceId(scope="system", code="number"),
            derivation_formula=f"average(map(Properties[Instrument/fitch/FitchRating]: {ratings_mapping}), map(Properties[Instrument/moodys/MoodysRating]: {ratings_mapping}), map(Properties[Instrument/sp/SPRating]: {ratings_mapping}), map(Properties[Instrument/internal_r/InternalRating]: {ratings_mapping}))",
        )
    )
    
    properties_response
except lusid.exceptions.ApiException as e:
    print(json.loads(e.body)["title"])

Error creating Property Definition 'Instrument/ratings-scope/AverageRating' because it already exists.


In [24]:
instruments_properties_response = api_factory.build(
        lusid.api.InstrumentsApi
    ).get_instruments(
        scope="default",
        identifier_type="LusidInstrumentId",
        request_body=instrument_response_objs,
        property_keys=[
            "Instrument/fitch/FitchRating",
            "Instrument/moodys/MoodysRating",
            "Instrument/sp/SPRating",
            "Instrument/internal_r/InternalRating",
            "Instrument/ratings-scope/AverageRating"
        ]
    )

inst_props_df = lusid_response_to_data_frame(instruments_properties_response.values)

instruments_properties_response


TypeError: Cannot map response object to pandas DataFrame or Series. The LUSID response object must have
                        either the values attribute or the to_dict() method, or be a list of objects with 
                        the to_dict() method